In [4]:
!pip install import-ipynb

**STEP1: Load google-drive, and libraries**

In [5]:
# Mounting google drive if it is already not mounted
def LoadGoogleDrive(googleDriveDir):
  # Link Google Drive
  import os
  # Check if Google Drive is already mounted
  if not os.path.exists(googleDriveDir + "/MyDrive"):
      print("Mounting Google Drive...")
      from google.colab import drive
      drive.mount(googleDriveDir)
  else:
      print("Google Drive is already mounted!")

from google.colab import drive
googleDriveFolder = "/content/drive"
LoadGoogleDrive(googleDriveFolder)

Google Drive is already mounted!


In [7]:
import sys
import import_ipynb

# 1. Set path to the directory where mylib.ipynb is located
lib_path = '/content/drive/MyDrive/Colab Notebooks/P_HINTS6/lib'
sys.path.append(lib_path)
# 2. Change current working directory to that path
%cd "$lib_path"

# 3. Now import the notebook
from app_lib import *


/content/drive/MyDrive/Colab Notebooks/P_HINTS6/lib


**STEP2: Load data**

In [8]:
ip_file_path = googleDriveFolder + "/MyDrive/Colab Notebooks/P_HINTS6/ip_data/hints6_public_filtered_v1.xlsx"
ip_sheet_name = 'hints6_public_colFilter'
df_orig = load_data(verbose_level=1, ip_file_path=ip_file_path, ip_sheet_name=ip_sheet_name)


Ip File Path: /content/drive/MyDrive/Colab Notebooks/P_HINTS6/ip_data/hints6_public_filtered_v1.xlsx
 SheetName: hints6_public_colFilter
df_orig.shape: (6252, 19)


In [9]:
df = df_orig.copy()

In [10]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Step 1: Make a copy of the original DataFrame
df = df_orig.copy()

# Step 2: Replace negative values with np.nan (only in numeric columns)
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].mask(df[col] < 0, np.nan)

# Step 3: Impute missing values using Random Forest
imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=20, random_state=42)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# ✅ df_imputed is your cleaned, imputed dataset


/usr/local/lib/python3.11/dist-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
import os

op_file_path = '/content/drive/MyDrive/Colab Notebooks/P_HINTS6/op_data/hints6_public_filtered_v2.xlsx'
sheet_name = 'df_imputed'

if os.path.exists(op_file_path):
    with pd.ExcelWriter(op_file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df_imputed.to_excel(writer, sheet_name=sheet_name, index=False)
else:
    df_imputed.to_excel(op_file_path, sheet_name=sheet_name, index=False)
